In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay
''
enable_replay(False)

set_defaults(
    cad_width=640, 
    height=480, 
)

import sys
sys.path.append('../')
from config import Config, Shape
from plate import get_base, get_screw_positions, get_key_positions, make_plate
#from case import make_case

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


/home/ben/anaconda3/envs/jcq3/lib/python3.10/site-packages/quaternion/calculus.py:310: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def fd_indefinite_integral(f, t):


In [2]:
rkn = [10,10,10,8]

config=Config(rkn, shape=Shape.LEAN)
kp = get_key_positions(config)

In [23]:
def make_case(config:Config) -> cq.Sketch:
    """TODO: add screw holes for microcontroller (check they are in right position!!)
            change fillet parameters
            add module for leds, encoder and aviator
            add screw holes"""
    
    # remove hard coding and put into Config!
    controller_y_offset = 5
    controllerBoxLength = config.controller.board_dimension_y - controller_y_offset
    controllerBoxWidth=config.controller.board_dimension_x + 5
    
    kp = get_key_positions(config)
    x_centre = 0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    case = case.push(list(kp.values()))
    
    
    case = (case.rect(foot_x+(config.caseGap+config.wallThickness)*2, 
                      foot_y+(config.caseGap+config.wallThickness)*2)
            .faces()
            .clean()
            .offset(config.wallThickness)
            .clean())
    
    
    case= (cq.Workplane()
           .placeSketch(case)
           .extrude(config.caseHeight))
    
    # add box for microcontroller
    case = (case
             .edges('>Y and <Z')
             .workplane(centerOption="CenterOfBoundBox", invert=False)
             .box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False],combine=False)
             .faces('<Z')
            .workplane(centerOption="CenterOfBoundBox")
            .tag("controllerBox")
            .union(case)
            )
    
    # tag outerfaces for future reference
    case.faces('not |Z').tag('outerfaces')

    # scoop out interior 
    case= (case.faces("+Z")
            .shell(-config.wallThickness)
            )
    
    # fillet edges
    case = (case.edges("|Z")
             .fillet(config.plateFillet))

    # set floor thickness 
    btm = case.faces('<Z[1]').clean().wires().toPending()
    case_without_btm=(btm.extrude(config.wallThickness, combine="cut"))
    new_btm = case.faces('<Z[1]').clean().wires().toPending().extrude(-config.floorThickness, combine=False).translate([0,0,-config.wallThickness])
    case = case_without_btm.union(new_btm)
    
    # add controllerbox holes.
    # TODO - remove hardcoding on screw hole size
#     case = (case.workplaneFromTagged("controllerBox")
#             .rect(config.controller.screw_hole_x,
#                    config.controller.screw_hole_y- config.wallThickness - controller_y_offset - (config.controller.board_dimension_y -config.controller.screw_hole_y)*0.5, 
#                    forConstruction=True ,centered=[True,True,True])
#             .vertices()
#             .cskHole(2.4, 4.8, 82, depth=None))
    
    
    return case




In [37]:

rkn = [10,10,10,8]

config=Config(rkn, shape=Shape.LEAN, floorThickness=10)
kp = get_key_positions(config)

case = make_case(config)

case
#case.faces(tag='outerfaces').edges('#Z and <Z').chamfer(1)


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [28]:
btm = case.faces('<Z[1]').clean().wires().toPending()

case_without_btm=(btm.extrude(config.wallThickness, combine="cut"))


new_btm = case.faces('<Z[1]').clean().wires().toPending().extrude(-config.floorThickness, combine=False).translate([0,0,-config.wallThickness])

case_without_btm.union(new_btm)



CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [56]:
cc= case.faces('<Z[1]').clean().wires().offset(0)#.extrude(10, combine=False)

cc
#cq.Workplane()#.placeSketch(cc)#.extrude(10)

AttributeError: 'Workplane' object has no attribute 'offset'

In [55]:
if True:
    base_fillet=2.5
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                      config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    base = cq.Sketch()
   

    base = base.push(kp.values())
    if config.shape == Shape.LEAN:
        base = base.rect(foot_x, foot_y)\
            .faces().clean().vertices().fillet(base_fillet).faces()\
            .wires().offset(0).clean()
    elif config.shape == Shape.HULL:
        base = base.rect(foot_x, foot_y)\
            .faces().hull().clean().wires().offset(12)
        
base

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.01s


CadViewerWidget(anchor=None, cad_width=640, glass=False, height=480, pinning=False, theme='light', title=None,…

In [ ]:
controllerBoxLength = 20
controllerBoxWidth=80

# find position of controller box
# add controller box corners to kp
if True:

    #x_centre = max(config.row_key_num)* config.rowSpacing*0.5
    x_centre = 0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    
    kp = get_key_positions(config)
    
    
    points_to_push = list(kp.values())
    case = case.push(points_to_push)

    
    case = (case.rect(foot_x+(config.caseGap+wall_thickness)*2, foot_y+(config.caseGap+wall_thickness)*2)
            .faces()
            .clean()
            .offset(config.wallThickness)
            .clean())
    case=cq.Workplane().placeSketch(case).extrude(config.caseHeight)
    case = (case
            .edges('>Y and <Z')
            .workplane(centerOption="CenterOfBoundBox")
            .box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False])
           )

    case=case.faces("+Z").shell(-wall_thickness).edges("|Z").fillet(3)

In [ ]:
case

In [ ]:
0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))

In [ ]:
controllerBoxLength = 20
controllerBoxWidth=80

# find position of controller box
# add controller box corners to kp
if True:

    #x_centre = max(config.row_key_num)* config.rowSpacing*0.5
    x_centre = 0.5*(min([x[0] for x in list(kp.values())])+ max([x[0] for x in list(kp.values())]))
    
    foot_x, foot_y = (config.columnSpacing / 2 + config.switchHoleSize, config.rowSpacing / 2 +
                          config.switchHoleSize) if config.shape == Shape.LEAN else (config.switchHoleSize, config.switchHoleSize)
    case = cq.Sketch()

    
    kp = get_key_positions(config)
    #points_to_push = [y[1] for y in list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))]
    points_to_push=[]
    for x in range(len(config.row_key_num)):
            points_to_push.append((x,0))
            points_to_push.append((x,config.row_key_num[x]-1))
    #points_to_push.extend([(x_centre+0.5*controllerBoxWidth,80) ,(x_centre-0.5*controllerBoxWidth,80), (x_centre,80)])
    points_to_push=[]
    for x in range(len(config.row_key_num)):
            points_to_push.append(kp[(x,0)])
            points_to_push.append(kp[(x,list(reversed(config.row_key_num))[x]-1)])
    case = case.push(points_to_push)
    #case = case.push([(0,0) ,(10,10)])
    
    case = (case.rect(foot_x+(config.caseGap+wall_thickness)*2, foot_y+(config.caseGap+wall_thickness)*2)
           .faces()
           .clean()
           .offset(config.wallThickness)
           .clean())
    case=cq.Workplane().placeSketch(case).extrude(config.caseHeight)

    #case=case.faces("+Z").shell(-wall_thickness).edges("|Z").fillet(3)
    
case

In [ ]:
len(config.row_key_num), config.row_key_num

In [ ]:
kp[len(config.row_key_num)-1, 3]


In [ ]:
[kp[len(config.row_key_num)-1, x] for x in range( 1, config.row_key_num[0])-1]

In [ ]:
points_to_remove = [kp[len(config.row_key_num)-1, x] for x in range(  1,config.row_key_num[0]-1)]

In [ ]:
idxs_to_drop = [(len(config.row_key_num)-1, x) for x in range(1, config.row_key_num[0]-1)]

idxs_to_drop

list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))

In [ ]:
points_to_push = [y[1] for y in list(filter( lambda x: x[0] not in idxs_to_drop, kp.items()))]
points_to_push.extend([(x_centre+0.5*controllerBoxWidth,80) ,(x_centre-0.5*controllerBoxWidth,80), (x_centre,80)])
    

In [ ]:
points_to_push

In [ ]:
[ [(x,) for x in range(len(config.row_key_num))] for y in [0, config.row_key_num[x]-1]]

In [ ]:
[[(j,i) for j in range(5)] for i in range(5)]

In [ ]:
[[(x,y) for y in [0, config.row_key_num[x]-1]] for x in range(len(config.row_key_num))]

In [ ]:
points_to_push=[]
for x in range(len(config.row_key_num)):
        points_to_push.append(kp[(x,0)])
        points_to_push.append(kp[(x,list(reversed(config.row_key_num))[x]-1)])

In [ ]:
points_to_push

In [ ]:
xx=(cq.Workplane('XY')
             .box(10,20, config.caseHeight, centered=[True,False,False])
             .pushPoints([(5,10)]).cskHole(2.4, 4.8, 82, depth=None)
)

xx

In [ ]:
controllerBoxLength = config.controller.board_dimension_y -5
controllerBoxWidth=config.controller.board_dimension_x + 5
(cq.Workplane('XZ')
.box(controllerBoxWidth,controllerBoxLength, config.caseHeight, centered=[True,False,False],combine=True)
 .faces("<Z")
    .workplane()
             .rect(config.controller.screw_hole_x-5,
                   config.controller.screw_hole_y, 
                   forConstruction=True )
            .vertices()
            .cskHole(2.2, 4.8, 82, depth=None)
)